In [1]:
import numpy as np
import pandas as pd
import os
import glob

import warnings
warnings.filterwarnings('ignore')

from sys import path
path.append("C:/Work/Github/rp_extract")

from FFmpeg import FFmpeg
from rp_extract_python import rp_extract

%load_ext autoreload
%autoreload 2

ImportError: No module named FFmpeg

In [2]:
DATA_DIR = "E:/Data/MIR/EU_SOUNDS"
FEATURE_DIR = "E:/Data/MIR/EU_SOUNDS_FEATURES"

In [3]:
mp3_decoder = FFmpeg("D:/Research/Tools/ffmpeg/bin/ffmpeg.exe")

In [4]:
f = "E:/Data/MIR/EU_SOUNDS/09301/00967B4452EDD93CB5847DAE2C681C2F3597469B.mp3"

samplerate, wavedata = mp3_decoder.convertAndRead(f)

print wavedata.shape

wavedata = wavedata / float(32768)

extracted_features = rp_extract(wavedata,                            # the two-channel wave-data of the audio-file
                                samplerate,                          # the samplerate of the audio-file
                                extract_rp          = True,          # <== extract this feature!
                                extract_rh          = True,
                                extract_ssd         = True,
                                extract_mvd         = True,
                                extract_trh         = True,
                                extract_tssd        = True,
                                transform_db        = True,          # apply psycho-accoustic transformation
                                transform_phon      = True,          # apply psycho-accoustic transformation
                                transform_sone      = True,          # apply psycho-accoustic transformation
                                fluctuation_strength_weighting=True, # apply psycho-accoustic transformation
                                skip_leadin_fadeout = 0,             # skip lead-in/fade-out. value = number of segments skipped
                                step_width          = 1)             # 

(4464000L,)


In [ ]:
dst_filename = "D:/tmp/eu_sounds_mp3_feature"

np.savez(dst_filename,
         rp = extracted_features["rp"],
         rh = extracted_features["rh"],
         trh = extracted_features["trh"],
         ssd = extracted_features["ssd"],
         tssd = extracted_features["tssd"],
         mvd = extracted_features["mvd"])

In [ ]:
i = 1

for dir_name in glob.glob(os.path.join(DATA_DIR, "*")):
    
    src_path = dir_name.replace("\\","/")
    dst_path = src_path.replace(DATA_DIR, FEATURE_DIR)
    
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    
    for mp3_name in glob.glob(os.path.join(src_path, "*.mp3")):
        
        src_filename = mp3_name.replace("\\","/")
        dst_filename = src_filename.replace(DATA_DIR, FEATURE_DIR)
        
        if os.path.exists("%s.npz" % (dst_filename)):
            i += 1
            continue
        
        try:
            
            samplerate, wavedata = mp3_decoder.convertAndRead(src_filename)
            wavedata = wavedata / float(32768)

            extracted_features = rp_extract(wavedata,                            # the two-channel wave-data of the audio-file
                                            samplerate,                          # the samplerate of the audio-file
                                            extract_rp          = True,          # <== extract this feature!
                                            extract_rh          = True,
                                            extract_ssd         = True,
                                            extract_mvd         = True,
                                            extract_trh         = True,
                                            extract_tssd        = True,
                                            transform_db        = True,          # apply psycho-accoustic transformation
                                            transform_phon      = True,          # apply psycho-accoustic transformation
                                            transform_sone      = True,          # apply psycho-accoustic transformation
                                            fluctuation_strength_weighting=True, # apply psycho-accoustic transformation
                                            skip_leadin_fadeout = 0,             # skip lead-in/fade-out. value = number of segments skipped
                                            step_width          = 1)             # 


            np.savez(dst_filename,
                     rp = extracted_features["rp"],
                     rh = extracted_features["rh"],
                     trh = extracted_features["trh"],
                     ssd = extracted_features["ssd"],
                     tssd = extracted_features["tssd"],
                     mvd = extracted_features["mvd"])
            
            i += 1
            
            if i % 10000 == 0:
                print i
            
        except Exception as e:
            #print src_filename, e
            pass
    